# 잠재 의미 분석(Latent Semantic Analysis, LSA)

## 특이값 분해(Singular Value Decomposition, SVD)
$$A=UΣV^\text{T} \ where \ A=M_{mn}$$ $$U=AA^\text{T}(\text{2x2}),Σ=\text{2x3}, V^\text{T}=A^\text{T}A(\text{3x3})$$

## 절단된 SVD(Truncated SVD)
일부 벡터를 삭제함 (=데이터의 차원을 줄임)
상대적으로 중요하지 않은 정보를 삭제함 <br>
-> 영상 처리: 노이즈 제거 / 자연어 처리: 설명력이 낮은 정보를 삭제하고 설명력이 높은 정보를 남김

대각 행렬 Σ의 대각 원소 값 중 상위 t개만 남김.<br>
U, V의 t열까지만 남김<br>
t: 찾고자하는 토픽의 수를 반영한 하이퍼파라미터 값.

## Full SVD

In [3]:
import numpy as np
A=np.array([[0,0,0,1,0,1,1,0,0],
            [0,0,0,1,1,0,1,0,0],
            [0,1,1,0,2,0,0,0,0],
            [1,0,0,0,0,0,0,1,1]]) # 과일이 / 길고 / 노란 / 먹고 / 바나나 / 사과 / 싶은 / 저는 / 좋아요
np.shape(A)

(4, 9)

In [6]:
U, s, VT = np.linalg.svd(A, full_matrices=True)
print(U.round(2))
print(np.shape(U))
print('\n')
print(s.round(2))
print(np.shape(s))
print('\n')
print(VT.round(2))
print(np.shape(VT))

[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
(4, 4)


[2.69 2.05 1.73 0.77]
(4,)


[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]
(9, 9)


In [ ]:
'''
    Numpy의 linalg.svd(): 특이값 분해의 결과로 특이값의 리스트를 반환함 (대각행렬X)
    따라서 S를 대각행렬로 변환해주어야 함.
'''

In [7]:
S = np.zeros((4, 9))
S[:4, :4] = np.diag(s) # np.diag: 대각행렬 생성 함수
print(S.round(2))
print(np.shape(S))

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]
(4, 9)


In [9]:
# U x S x VT = A 확인
np.allclose(A, np.dot(U, np.dot(S, VT)).round(2))

True

## Truncated SVD

In [11]:
S = S[:2, :2]
print(S.round(2))
print('\n')

U = U[:, :2]
print(U.round(2))
print('\n')

VT = VT[:2, :]
print(VT.round(2))

[[2.69 0.  ]
 [0.   2.05]]


[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


In [12]:
# Truncated SVD는 U x S x VT != A임.
A_prime = np.dot(np.dot(U, S), VT)
print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


In [ ]:
'''
    U: 4x2 size
        문서의 개수 x 토픽의 수 t와 동일함.
        단어의 개수인 9는 유지되지 않으나 문서의 개수인 4의 크기가 유지되었으니 4개 문서 각각을 2개의 값으로 표현한 것.
        즉, U의 각 행은 잠재 의미를 표현하기 위한 수치화된 각각의 문서 벡터로 볼 수 있음.
    VT: 2x9 size
        토픽의 수 t x 단어의 개수의 크기와 동일함.
        VT의 각 열은 잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터로 볼 수 있음.
'''